In [1]:
import pandas as pd
import datetime
import numpy as np
import re
import seaborn as sb
import matplotlib as mlp
import matplotlib.pyplot as plt
from hashlib import sha256

%matplotlib inline

In [2]:
pd.options.display.max_columns=1000
mlp.rcParams["figure.dpi"] = 200
mlp.rcParams["figure.facecolor"] = "white"

In [5]:
vinculacion = pd.read_excel("../../../../Outcome Measurement Data/HIV/SIGPRO_Raw/sigpro_f4_JanNov2018.xlsx", 4)
psico = pd.read_excel("../../../../Outcome Measurement Data/HIV/SIGPRO_Raw/sigpro_f4_JanNov2018.xlsx", 8)
its = pd.read_excel("../../../../Outcome Measurement Data/HIV/SIGPRO_Raw/sigpro_f4_JanNov2018.xlsx", 6)
aa = pd.read_excel("../../../../Outcome Measurement Data/HIV/SIGPRO_Raw/sigpro_f4_JanNov2018.xlsx", 7)

In [22]:
pb_tvc = pd.read_excel("../../../../Outcome Measurement Data/HIV/SIGPRO_Raw/sigpro_f4_JanNov2018.xlsx", 5)

In [8]:
# Unique IDs begin with a letter indicating either Male, Female or Trans. 
# Continue with 6 numbers indicating date of birth. Are followed by 
# 2 unknown numbers and 3 numbers standing for the municipality code, 
# then 4 or 3 letters that stand for the initials of the patient. 
# Finally two numbers of unknown meaning. Must convert dates to age,
# obtain sex column and unique hash

In [21]:
def FuncTry(f, debug_prefix = "DEBUG"):
    def wrapper(data):
        try:
            return f(data)
        except:
            print(debug_prefix + ": Bad data:", data)
            return np.NaN
    return wrapper

def parseDate(input_str, format_ = "%d%m%y"):
    try:
        return datetime.datetime.strptime(input_str[1:7], "%d%m%y")
    except:
        print("DATE: Bad date", input_str)
        return pd.NaT

muni_expr = re.compile(r"(\d{4})[A-Z]+")
def extractMunicode(input_str):
    try:
        return int(muni_expr.search(input_str).group(1))
    except:
        print("Bad municode:", input_str)
        return 0
    
def getAgeFuncGen(cui_field):
    def getAge(row):
        try:
            return np.floor((row.Date - parseDate(row[cui_field]).date()).days/365.25)
        except:
            print("AGE: Bad dates:", row.Date, row[cui_field])
            return np.NaN
    return getAge
        
def processDatabase(df, cui_field = "codigounico"):
    df["Gender"]  = df[cui_field].map(lambda x: x[0:1] if type(x) == str else np.NaN)
    df["Age"]     = df.apply(getAgeFuncGen(cui_field), axis=1)
    df["municode"]= df[cui_field].map(extractMunicode)
    hashField(df, cui_field)
    
def hashField(df, hash_field):
    df[hash_field]     = df[hash_field].map(
        FuncTry(lambda x: sha256(x.encode()).hexdigest(), "HASH"))

In [23]:
pb_tvc["Date"]    = pd.to_datetime(pb_tvc.fechareal, format = "%d/%m/%Y").map(lambda x: x.date())
processDatabase(pb_tvc)
hashField(pb_tvc, "responsable")
hashField(pb_tvc, "educador")

AGE: Bad dates: NaT T07077811217DEPE00
AGE: Bad dates: NaT T20029911107MIRA00
AGE: Bad dates: NaT T31087949999ALAL00
DATE: Bad date T01258911801BEMO00
DATE: Bad date M113/012001MASO00
AGE: Bad dates: NaT M03099210101LEME00
AGE: Bad dates: NaT T20049111601PATZ00
AGE: Bad dates: NaT T18029410101NICA00
AGE: Bad dates: NaT T04018410101JUAV00
AGE: Bad dates: NaT T08017812102VIVE00
AGE: Bad dates: NaT T17066911904DELI00
AGE: Bad dates: NaT T01015010101CUCA00
AGE: Bad dates: NaT T24099210101SAPE00
AGE: Bad dates: NaT T12099111801FEAL00
AGE: Bad dates: NaT T14089010108SEGO00
AGE: Bad dates: NaT T14089810108MAGO00
AGE: Bad dates: NaT T15039010101MACA00
AGE: Bad dates: NaT T25049010108CAGU00
AGE: Bad dates: NaT T22049359999SAGO00
AGE: Bad dates: NaT T15039310101DAQU00
AGE: Bad dates: NaT T05118610101DADU00
AGE: Bad dates: NaT T14129011501ROSU00
AGE: Bad dates: NaT T16079110301JIED00
AGE: Bad dates: NaT T31018810101BRGO00
AGE: Bad dates: NaT T12028810101JOPE00
AGE: Bad dates: NaT T12017810504MALO

In [33]:
pb_tvc.to_csv("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018 - PB_TVC.csv")

In [34]:
its["Date"]    = its.apply(lambda x: datetime.date(int(x.codanno+2016), int(x.codmes), 1) if not np.isnan(x.codanno) else pd.NaT, axis=1)
#its.iloc[6958]
its.Date.head(2)

0    2018-01-01
1    2018-01-01
Name: Date, dtype: object

In [35]:
#np.floor((its.Date - 
r = its.iloc[1]
(r.Date - parseDate(r["codigounico"]).date()).days/365.25

38.04517453798768

In [36]:
processDatabase(its)
hashField(its, "responsable")
its.to_csv("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018 - ITS.csv")

DATE: Bad date M01/09912214ELRE00
AGE: Bad dates: NaT M15116910501RUBA01
AGE: Bad dates: NaT T05099229999JOSA00
AGE: Bad dates: NaT T12099411002LIMO00
DATE: Bad date M01/09912214ELRE00
AGE: Bad dates: NaT T05099229999JOSA00
AGE: Bad dates: NaT T12099411002LIMO00
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan


In [37]:
aa["Date"]    = aa.apply(lambda x: datetime.date(int(x.codanno+2016), int(x.codmes), 1) if not np.isnan(x.codanno) else pd.NaT, axis=1)
aa.Date.head(2)

0    2018-01-01
1    2018-01-01
Name: Date, dtype: object

In [38]:
aa.codigounico.map(lambda x: type(x)).value_counts()

<class 'str'>      17641
<class 'float'>        1
Name: codigounico, dtype: int64

In [39]:
processDatabase(aa)
hashField(aa, "responsable")
aa.to_csv("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018 - AA.csv")

DATE: Bad date M01/09912214ELRE00
AGE: Bad dates: NaT M15116910501RUBA01
DATE: Bad date nan
DATE: Bad date HSH MAYORES DE 18 AÃ‘OS
AGE: Bad dates: NaT T05099229999JOSA00
DATE: Bad date TRANS TRABAJADORAS SEXUALES
AGE: Bad dates: NaT T12099411002LIMO00
DATE: Bad date TRANS MAYORES DE 18 AÃ‘OS
DATE: Bad date M01/09912214ELRE00
AGE: Bad dates: NaT T05099229999JOSA00
DATE: Bad date TRANS TRABAJADORAS SEXUALES
AGE: Bad dates: NaT T12099411002LIMO00
DATE: Bad date TRANS MAYORES DE 18 AÃ‘OS
Bad municode: nan
Bad municode: HSH MAYORES DE 18 AÃ‘OS
Bad municode: TRANS TRABAJADORAS SEXUALES
Bad municode: TRANS MAYORES DE 18 AÃ‘OS
Bad municode: TRANS TRABAJADORAS SEXUALES
Bad municode: TRANS MAYORES DE 18 AÃ‘OS
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad d

In [40]:
psico["Date"] = psico.apply(lambda x: datetime.date(int(x.codanno+2016), int(x.codmes), 1) if not np.isnan(x.codanno) else pd.NaT, axis=1)
psico.Date.head(2)

0    2018-04-01
1    2018-04-01
Name: Date, dtype: object

In [41]:
processDatabase(psico)
hashField(psico, "responsable")
psico.to_csv("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018 - Psico.csv")

In [42]:
vinculacion["Date"] = vinculacion.apply(lambda x: datetime.date(int(x.codanno+2016), int(x.codmes), 1) if not np.isnan(x.codanno) else pd.NaT, axis=1)
vinculacion.Date.head(2)

0    2018-02-01
1    2018-03-01
Name: Date, dtype: object

In [43]:
processDatabase(vinculacion)
hashField(vinculacion, "responsable")
vinculacion.to_csv("../../../../Outcome Measurement Data/HIV/SIGPRO/sigpro_f4_JanNov2018 - Vinculacion.csv")

HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan
HASH: Bad data: nan


# Some exploring:


In [24]:
vinculacion.Date.map(lambda x: x.year*100+x.month).value_counts().sort_index()

201801    23
201802    37
201803    24
201804    61
201805    65
201806    51
201807    59
201808    79
201809    72
201810    56
201811    54
Name: Date, dtype: int64